In [1]:
# installing all the required libraries
!pip install landsatxplore folium rasterio geopandas pylandtemp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of geopandas to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fiona to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.1 MB/s eta 

The landsatxplore library is a Python package designed to facilitate the search
and download of Landsat satellite imagery from the USGS EarthExplorer portal.
This library allows users to programmatically search for Landsat data based on
specific criteria such as date range,cloud cover percentage, location, and specific
Landsat missions (e.g., Landsat 8, Landsat 9). It also provides an interface to
download the selected imagery directly, simplifying the process of acquiring
satellite data for analysis.

1. rasterio is used to read the rasters
2. numpy is often used in processing raster data for operations like mathematical transformations or array manipulations.
3. matplotlib for visullisation

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Mounting the google drive: In order to add the shapefile of our Area of interest(NCT of Delhi in this case), that is saved in Google Drive.

In [5]:
# this code prompts user to enter your USGS credentials
# allows to select a scene with date info

from landsatxplore.api import API
from landsatxplore.earthexplorer import EarthExplorer
import getpass
import os
from datetime import datetime, timedelta

# Prompt user for username and password (password input is hidden)
username = input("Enter your EarthExplorer username: ")
password = getpass.getpass("Enter your EarthExplorer password: ")

# Initialize a new API instance and get an API key
api = API(username, password)

# Search for Landsat 8/9 scenes in the specified date range
scenes = api.search(
    dataset='landsat_ot_c2_l1',  # dataset identifier for Landsat 8/9 Collection 2 Level 1 data
    latitude=28.6139,
    longitude=77.2090,
    start_date='2023-05-15',
    end_date='2023-06-15',
    max_cloud_cover=20
)

# Print the number of scenes found
print(f"{len(scenes)} scenes found.")

# Function to convert Julian Day to a date
def julian_day_to_date(year, julian_day):
    return datetime(year, 1, 1) + timedelta(days=julian_day - 1)

# Display the list of scenes for user selection
print("\nAvailable Scenes:")
for index, scene in enumerate(scenes):
    display_id = scene.get('displayId', scene['entity_id'])

    # Extract the year and Julian Day from displayId
    year = int(display_id[9:13])
    julian_day = int(display_id[13:16])

    # Convert Julian Day to standard date
    acquisition_date = julian_day_to_date(year, julian_day).strftime('%Y-%m-%d')

    print(f"{index + 1}. {display_id} - {acquisition_date}")

# Prompt the user to select which scenes to download
selected_indices = input("\nEnter the numbers of the scenes you want to download (comma-separated): ")
selected_indices = [int(i.strip()) - 1 for i in selected_indices.split(',')]

# Initialize a new EarthExplorer instance
ee = EarthExplorer(username, password)

# Download each selected scene
output_dir = '/content/landsat8_data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for index in selected_indices:
    scene = scenes[index]
    entity_id = scene['entity_id']
    display_id = scene.get('displayId', entity_id)

    print(f"Downloading all files for {display_id}...")

    # Download the entire scene
    ee.download(entity_id, output_dir)

# Logout from EarthExplorer
ee.logout()

# Logout from the API
api.logout()

Enter your EarthExplorer username: adscorpmail
Enter your EarthExplorer password: ··········
3 scenes found.

Available Scenes:
1. LC81460402023161LGN00 - 2023-06-10
2. LC91470402023160LGN03 - 2023-06-09
3. LC91460402023153LGN00 - 2023-06-02

Enter the numbers of the scenes you want to download (comma-separated): 3
Download failed with dataset id 1 of 2. Re-trying with the next one.


100%|██████████| 1.22G/1.22G [01:19<00:00, 16.5MB/s]


This script allows users to download Landsat data from the USGS EarthExplorer platform. It begins by prompting for user credentials, a date range, maximum cloud cover, and geographical coordinates. After initializing the API and searching for relevant scenes based on these inputs, it displays the available scenes for the user to select from. The user then chooses which scenes to download. The script downloads the selected scenes to a specified directory, and finally, it logs out of both the EarthExplorer and API sessions to ensure secure termination.

Note: change the values of `latitude,longitude,start_date,end_date,max_cloud_cover` as per your requirement.

In [6]:

# for extracting tar file in the colab itself

import tarfile
import os

# Define the path to the .tar file
tar_file_path = '/content/landsat8_data/LC09_L1TP_146040_20230602_20230602_02_T1.tar'

# Define the extraction directory
extraction_path = '/content/extracted_files/'

# Create the extraction directory if it doesn't exist
if not os.path.exists(extraction_path):
    os.makedirs(extraction_path)

# Open the .tar file and extract all its contents
with tarfile.open(tar_file_path, 'r') as tar_file:
    tar_file.extractall(path=extraction_path)

print(f'Files extracted to {extraction_path}')


Files extracted to /content/extracted_files/


This code extracts the contents of a .tar file in Google Colab. It imports the tarfile and os modules to handle file extraction and directory operations. The path to the .tar file (tar_file_path) and the extraction directory (extraction_path) are defined. The script checks if the extraction directory exists, creating it if necessary. It then opens the .tar file in read mode and extracts all files to the specified directory using tarfile.open() and extractall(). Finally, it prints a message confirming the files have been extracted to the specified location.

Note: before executing the code, copy the file path of .tar file dowloaded in the landsat folder, and paste it in the `tar_file_path`.